In [1]:
# # # # Install Hugging Face libraries
# %pip install  --upgrade \
#   "evaluate" \
#   "tensorboard" \
#   "flash-attn" \
#   "liger-kernel" \
#   "setuptools" \
#   "deepspeed" \
#   "lm-eval[api]" \
#   "torch"\
#   "torchvision" \
#   "transformers" \
#   "datasets" \
#   "accelerate" \
#   "bitsandbytes" \
#   "trl" \
#   "peft" \
#   "lighteval" \
#   "hf-transfer"

### Import libraries and frameworks

In [2]:
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer, set_seed, BitsAndBytesConfig
from transformers.trainer_utils import get_last_checkpoint
from transformers.utils import is_liger_kernel_available
from datasets import load_dataset
from trl import SFTTrainer, TrlParser, ModelConfig, SFTConfig, get_peft_config
from peft import AutoPeftModelForCausalLM

import pandas as pd
from datasets import Dataset, DatasetDict

In [3]:
import os
os.environ["TOKENIZERS_PARALLELISM"] = "false"
os.environ["PYTORCH_CUDA_ALLOC_CONF"] = "expandable_segments:True"

### Check device

In [4]:
device = torch.accelerator.current_accelerator().type if hasattr(torch, "accelerator") else "cuda"
print(f"Device: {device}")

Device: cuda


In [5]:
# Use BitsAndBytesConfig for quantization that helps to reduce model size
from transformers import BitsAndBytesConfig

### Process dataset

In [6]:
from datasets import load_from_disk

In [7]:
processed_data_path = "./data/processed/radiology_datasets"
dataset = load_from_disk(processed_data_path)

In [9]:
dataset

DatasetDict({
    train: Dataset({
        features: ['findings', 'impression', 'text', 'clinic_id', 'modality', 'clinic_modality'],
        num_rows: 8865
    })
    validation: Dataset({
        features: ['findings', 'impression', 'text', 'clinic_id', 'modality', 'clinic_modality'],
        num_rows: 1901
    })
    test: Dataset({
        features: ['findings', 'impression', 'text', 'clinic_id', 'modality', 'clinic_modality'],
        num_rows: 1915
    })
})

### Load model

In [10]:
# 1. APPLY QUANTIZATION (This was missing!)
nf4_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_use_double_quant=True,
    bnb_4bit_compute_dtype=torch.bfloat16  # Use bfloat16 for better memory efficiency
)

# %%
# Load model WITH quantization
model_name = "microsoft/MediPhi-Instruct"
model = AutoModelForCausalLM.from_pretrained(
    model_name,
    quantization_config=nf4_config,  
    dtype=torch.bfloat16,      # Use bfloat16 for memory efficiency
    device_map="auto",               # Automatically distribute across GPUs
    trust_remote_code=True
)

tokenizer = AutoTokenizer.from_pretrained(
    model_name,
    trust_remote_code=True,
    padding_side="right"  # Ensure consistent padding
)

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [11]:
# model_name = "microsoft/MediPhi-Instruct"
# model = AutoModelForCausalLM.from_pretrained(model_name)
# tokenizer = AutoTokenizer.from_pretrained(model_name)

### Understand model architechture

In [12]:
model

Phi3ForCausalLM(
  (model): Phi3Model(
    (embed_tokens): Embedding(32064, 3072, padding_idx=32000)
    (layers): ModuleList(
      (0-31): 32 x Phi3DecoderLayer(
        (self_attn): Phi3Attention(
          (o_proj): Linear4bit(in_features=3072, out_features=3072, bias=False)
          (qkv_proj): Linear4bit(in_features=3072, out_features=9216, bias=False)
        )
        (mlp): Phi3MLP(
          (gate_up_proj): Linear4bit(in_features=3072, out_features=16384, bias=False)
          (down_proj): Linear4bit(in_features=8192, out_features=3072, bias=False)
          (activation_fn): SiLU()
        )
        (input_layernorm): Phi3RMSNorm((3072,), eps=1e-05)
        (post_attention_layernorm): Phi3RMSNorm((3072,), eps=1e-05)
        (resid_attn_dropout): Dropout(p=0.0, inplace=False)
        (resid_mlp_dropout): Dropout(p=0.0, inplace=False)
      )
    )
    (norm): Phi3RMSNorm((3072,), eps=1e-05)
    (rotary_emb): Phi3RotaryEmbedding()
  )
  (lm_head): Linear(in_features=3072, out_

In [13]:
trainable_params = 0
all_param = 0
for _, param in model.named_parameters():
    all_param += param.numel()
    if param.requires_grad:
        trainable_params += param.numel()
print(
    f"trainable params: {trainable_params} || "
    f"all params: {all_param} || "
    f"trainable%: {100 * trainable_params / all_param:.2f}%"
)

trainable params: 197200896 || all params: 2009140224 || trainable%: 9.82%


### Before training test the model

In [14]:
from transformers import pipeline, StoppingCriteria

In [15]:
findings = dataset['test'][0]['findings']
impression = dataset['test'][0]['impression']

print(f"Findings: {findings}\n")
print(f"Impressions: {impression}")

Findings: [CLINIC: clinic_1] [MODALITY: MR] FINDINGS: No abnormality along the sacral plexus presacral. Left sciatic nerve is normal in the greater sciatic foramen to the mid thigh. There is no mass or compression or edema along this nerve. Beginning distal mid thigh axial 36 and 35, there is intense edema of the peroneal branch of the sciatic nerve extending to the inferior margin lateral femoral condyle level left side only. There is no soft tissue mass or cyst along this nerve. No notable edema at the level of the fibular head. There is muscle denervation edema anterior and peroneal muscle compartments of the proximal leg left side only. No abnormality along the tibiofibular joint and specifically no marginating cyst. There is no other muscle denervation edema. The tibial branch of this left sciatic nerve is normal. Remaining muscles are normal. Left hamstring origin is intact with no marginating inflammation. Sacrum and sacroiliac joints are normal. Pubic symphysis has no marginati

In [16]:
# Check what token ID 32007 represents
print(f"Token 32007: '{tokenizer.decode([32007])}'")

Token 32007: '<|end|>'


In [17]:
# https://huggingface.co/microsoft/MediPhi-Instruct
# Radiology-specific system message
system_message = """You are an expert radiologist assistant specializing in generating accurate and concise medical impressions from radiology
       findings.
    
      Your task is to:
      1. **Analyze the findings**: Carefully review all clinical findings, history, and technique information
      2. **Generate focused impressions**: Create clear, prioritized conclusions that directly address the clinical question
      3. **Maintain clinical accuracy**: Ensure all significant findings are appropriately characterized
      4. **Use appropriate medical terminology**: Follow standard radiological reporting conventions
      5. **Adapt communication style**: Match the institutional reporting style and level of detail expected
    
      Generate only the IMPRESSION section based on the provided clinical information."""

# Hugging Face pipeline for text generation does apply apply_chat_template under the hood. 
# So we do not need to process for the text generation
messages = [
    {"role": "system", "content": system_message},
    {"role": "user", "content": findings},
]

pipe = pipeline(
    "text-generation",
    model=model,
    tokenizer=tokenizer,
)

#  stops generation when the model generates token ID 32007
class EosListStoppingCriteria(StoppingCriteria):
  def __init__(self, eos_sequence = [32007]):
      self.eos_sequence = eos_sequence

  def __call__(self, input_ids: torch.LongTensor, scores: torch.FloatTensor, **kwargs) -> bool:
      last_ids = input_ids[:,-len(self.eos_sequence):].tolist()
      return self.eos_sequence in last_ids

generation_args = {
    "max_new_tokens": 200,
    "return_full_text": False,
    "temperature": 0.0,
    "do_sample": False,
    "stopping_criteria": [EosListStoppingCriteria()]

}
output = pipe(messages, **generation_args)
print(f"AI: {output[0]['generated_text']}")

Device set to use cuda:0
The following generation flags are not valid and may be ignored: ['temperature']. Set `TRANSFORMERS_VERBOSITY=info` for more details.


AI:  IMPRESSION:

The MR imaging reveals intense edema of the peroneal branch of the left sciatic nerve extending to the inferior margin of the lateral femoral condyle, with no associated soft tissue mass or cyst. There is also muscle denervation edema in the anterior and peroneal muscle compartments of the proximal left leg. The tibiofibular joint and the left hip joint are normal, with no evidence of effusion or subchondral edema. The right superior pubic ramus shows a healed fracture with deformity, and the right inferior pubic ramus has mild deformity from a prior healed fracture. The right hip joint has moderate effusion, while the left hip joint has no effusion. There is mild chondral thinning on the right side of the hip joint, with no acute chondral


In [18]:
print(f"{impression}")

1. Long segment intense neuritis with edema peroneal branch left sciatic nerve beginning distal mid femoral shaft and extending to the knee joint margin. Associated denervation edema anterior and peroneal muscle compartments of the proximal left leg. No mass or cyst or compression along this nerve. No other abnormality along the left sciatic nerve.


In [19]:
# From the above testing, it is clear that Medphi is generating more or less similar text generation.
# WIth fine tiuning the model might learn more numances of the dataset provided. 

### Model training

In [20]:
from transformers import BitsAndBytesConfig
import torch
from trl import SFTTrainer, SFTConfig
from peft import LoraConfig, get_peft_model
from peft.optimizers import create_lorafa_optimizer

In [21]:
lora_config = LoraConfig(
    r=8,
    lora_alpha=32,
    bias="none",
    target_modules = ['o_proj', 'qkv_proj', 'gate_up_proj', 'down_proj'],
    task_type="CAUSAL_LM"
)

In [22]:
peft_model = get_peft_model(model, lora_config)

In [23]:
peft_model.print_trainable_parameters()

trainable params: 12,582,912 || all params: 3,833,662,464 || trainable%: 0.3282


In [24]:
import evaluate
import numpy as np
from sklearn.metrics import accuracy_score

def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    metrics = {}
    
    # Decode in smaller chunks to save memory
    batch_size = 32  # Adjust based on your GPU memory
    decoded_preds = []
    decoded_labels = []
    
    for i in range(0, len(predictions), batch_size):
        batch_preds = predictions[i:i+batch_size]
        batch_labels = labels[i:i+batch_size]
        decoded_preds.extend(tokenizer.batch_decode(batch_preds, skip_special_tokens=True))
        decoded_labels.extend(tokenizer.batch_decode(batch_labels, skip_special_tokens=True))
        # Free memory
        del batch_preds, batch_labels
        torch.cuda.empty_cache()
    
    # Compute ROUGE
    rouge = evaluate.load("rouge")
    rouge_score = rouge.compute(predictions=decoded_preds, references=decoded_labels)
    metrics.update(rouge_score)
    
    return metrics

In [29]:
from transformers import TrainingArguments
from trl import SFTTrainer, SFTConfig
import os 

sft_config = SFTConfig(
    # Basic training parameters
    output_dir="./results",
    num_train_epochs=3,
    per_device_train_batch_size=4,
    per_device_eval_batch_size=1,
    gradient_accumulation_steps=8,
    
    # Optimization
    learning_rate=2e-4,
    weight_decay=0.001,
    warmup_ratio=0.03,
    lr_scheduler_type="cosine",
    optim="adamw_torch",
    
    # Evaluation and saving
    eval_strategy="steps",
    eval_steps=20,
    save_strategy="steps",
    save_steps=20,
    save_total_limit=2,
    greater_is_better=False,
    load_best_model_at_end=True,
    metric_for_best_model="eval_loss",
    
    # Logging
    logging_dir="./logs",
    logging_strategy="steps",
    logging_steps=20,

    # assistant_only_loss=True,

    # Memory and performance
    dataloader_drop_last=True,
    dataloader_num_workers=4,
    remove_unused_columns=False,
    
    # Mixed precision training
    bf16=True if torch.cuda.is_bf16_supported() else False,
    
    # SFT-specific parameters
    max_length=1024,
    packing=True,  # Pack multiple short sequences into one
    dataset_text_field="text",
    
    # Gradient settings
    max_grad_norm=0.3,
    gradient_checkpointing=True,
)

In [30]:
# Create Trainer object
trainer = SFTTrainer(
    model=peft_model,
    args=sft_config,
    train_dataset=dataset['train'],
    eval_dataset=dataset['validation'],
    peft_config=lora_config,
    compute_metrics=compute_metrics
)

TypeError: SFTTrainer.__init__() got an unexpected keyword argument 'attn_implementation'

In [27]:
train_result = trainer.train()

OutOfMemoryError: CUDA out of memory. Tried to allocate 444.00 MiB. GPU 0 has a total capacity of 23.53 GiB of which 48.69 MiB is free. Including non-PyTorch memory, this process has 23.47 GiB memory in use. Of the allocated memory 22.63 GiB is allocated by PyTorch, and 393.33 MiB is reserved by PyTorch but unallocated. If reserved but unallocated memory is large try setting PYTORCH_CUDA_ALLOC_CONF=expandable_segments:True to avoid fragmentation.  See documentation for Memory Management  (https://pytorch.org/docs/stable/notes/cuda.html#environment-variables)

### Save the adapter

In [ ]:
def save_lora_adapter(trainer, save_path="./lora_adapter"):
    
    # Save the adapter
    trainer.model.save_pretrained(save_path)
    trainer.tokenizer.save_pretrained(save_path)
    
    print(f"LoRA adapter saved to: {save_path}")
    print(f"Adapter size: {get_directory_size(save_path):.2f} MB")
    
    return save_path

In [ ]:
def get_directory_size(path):
    total = 0
    for dirpath, dirnames, filenames in os.walk(path):
        for f in filenames:
            fp = os.path.join(dirpath, f)
            total += os.path.getsize(fp)
    return total / (1024 * 1024)

In [ ]:
adapter_path = save_lora_adapter(trainer)

## Push to Huggingface

In [ ]:
from huggingface_hub import HfApi, create_repo

In [ ]:
token = 'removed it'

In [ ]:
api = HfApi(token=token)

In [ ]:
repo_id="sabber/medphi-medical-qa-adapter"

In [ ]:
create_repo(repo_id=repo_id, token=token, exist_ok=True)

In [ ]:
files_to_upload = [
    "adapter_config.json",
    "adapter_model.safetensors",  # or adapter_model.bin
    "tokenizer.json",
    "tokenizer_config.json",
    "special_tokens_map.json"
]

for file in files_to_upload:
    file_path = os.path.join(adapter_path, file)
    if os.path.exists(file_path):
        api.upload_file(
            path_or_fileobj=file_path,
            path_in_repo=file,
            repo_id=repo_id,
            token=token
        )
        print(f"Uploaded: {file}")

### Test model with trained adapter

In [ ]:
os.listdir("./")

In [ ]:
os.listdir("./lora_adapter")

In [ ]:
import json

In [ ]:
# 1. First, let's check the current adapter config
with open("./lora_adapter/adapter_config.json", "r") as f:
    adapter_config = json.load(f)

print("Current adapter config:")
print(adapter_config)

# 2. Add the missing base model path if it's not there
if "base_model_name_or_path" not in adapter_config or adapter_config["base_model_name_or_path"] is None:
    adapter_config["base_model_name_or_path"] = "microsoft/MediPhi-Instruct"
    
    # Save the fixed config
    with open("./lora_adapter/adapter_config.json", "w") as f:
        json.dump(adapter_config, f, indent=2)
    
    print("✅ Fixed adapter_config.json with base model path")
else:
    print("✅ Base model path already exists")

In [ ]:
from peft import AutoPeftModelForCausalLM
from transformers import AutoTokenizer

ft_model = AutoPeftModelForCausalLM.from_pretrained(
    "./lora_adapter",
    torch_dtype="auto",
    device_map="auto"
)
ft_tokenizer = AutoTokenizer.from_pretrained("./lora_adapter")